In [2]:
import numpy as np
import math
import random as rn
import pandas as pd
from library import Map, GradientDescent
import time
from random import randrange

#profiling libraries
import cProfile
import pstats

In [3]:
'''REGRESSION'''

'''Downloading dataset'''
dataset = pd.read_csv("/Users/alessandrococcia/Downloads/ObesityDataSet.csv")

'''Dataset sampling'''
dataset = dataset.sample(frac=1, ignore_index=True) #shuffle sample in the training set

'''mapping delle stringhe'''
dataset = pd.get_dummies(dataset, drop_first=True, dtype=float)

'''normalization'''
dataset = (dataset-dataset.mean())/dataset.std()

'''Inserimento colonna di bias'''
dataset.insert(0, "Bias", np.ones(len(dataset)), True) #Bias row


TRAIN_TEST_SPLIT_PERCENTAGE = 0.9
dataset_training = dataset[:int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE)]
dataset_test = dataset[int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE):]


In [144]:
'''CLASSIFICATION'''

'''Downloading dataset'''
dataset = pd.read_csv("/Users/alessandrococcia/Downloads/ObesityDataSet.csv")

'''Dataset sampling'''
dataset = dataset.sample(frac=1, ignore_index=True) #shuffle sample in the training set

'''mapping delle stringhe'''
dataset.replace(to_replace=("Insufficient_Weight", "Normal_Weight", "Overweight_Level_I", "Overweight_Level_II"), value=0, inplace=True)
dataset.replace(to_replace=("Obesity_Type_I", "Obesity_Type_II", "Obesity_Type_III"), value=1, inplace=True)
dataset = pd.get_dummies(dataset, drop_first=True, dtype=float)

'''normalization'''
dataset = (dataset-dataset.mean())/dataset.std()

'''Inserimento colonna di bias'''
dataset.insert(0, "Bias", np.ones(len(dataset)), True) #Bias row


TRAIN_TEST_SPLIT_PERCENTAGE = 0.9
dataset_training = dataset[:int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE)]
dataset_test = dataset[int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE):]

/var/folders/yq/gyf11wn125qgkzlcv24mbp3w0000gn/T/ipykernel_67114/3248209574.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace(to_replace=("Obesity_Type_I", "Obesity_Type_II", "Obesity_Type_III"), value=1, inplace=True)


In [145]:
'''CLASSIFICATION'''

output_string = 'NObeyesdad'
input_totale = dataset_training.loc[:len(dataset_training), dataset_training.columns != output_string]
output_totale = dataset_training.loc[:len(dataset_training), dataset_training.columns == output_string]

'''Values to determine threshold'''
positive_value, negative_value = max(output_totale['NObeyesdad'].unique()), min(output_totale['NObeyesdad'].unique())
threshold = 0.5

class NeuralNetwork:
    def __init__(self, learning_rate, len_input, num_layers, num_layer_nodes, epochs=1):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.len_input = len_input
        self.num_layers = num_layers
        self.num_layer_nodes = num_layer_nodes
        self.weight_list = []
        self.layer_list = []
        self.derivate_list = []
        self.threshold = 0.5
        self.totalerror = 0


        #inizializzazione matrice dei pesi
        for i in range(self.num_layers-1):
            new_matrix = np.random.rand(self.num_layer_nodes[i], self.num_layer_nodes[i+1]) #matrice con righe=i e colonne i+1: (input,output) di un layer
            self.weight_list.append(new_matrix)

        #inizializzazione layer
        for i in range(self.num_layers):
            self.layer_list.append(np.zeros(self.num_layer_nodes[i]))

        #inizializzazione lista derivate parziali
        for i in range(self.num_layers-1): #non viene contato il layer di input
            self.derivate_list.append(np.zeros(self.num_layer_nodes[i+1]))

    def activation(self, x, type='sigmoid'): 
        if type=='sigmoid':
            return 1/(1+np.exp(-x))
        elif type=='tanh':
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        return None

    def activation_derivative(self, x, type='sigmoid'):
        if type=='sigmoid':
            return (1-self.activation(x, type='sigmoid'))*self.activation(x, type='sigmoid')
        return None
    
    def training(self, dataset_training):
        for e in range(self.epochs):
            print("Epoca numero", e)
            for i in range(dataset_training.shape[0]): #ciclo che itera per ogni sample del dataset 
                sample = dataset_training.loc[i:i]
                self.feedforwarding(sample)
                #print("Error: ", self.totalerror)
                self.backpropagation(i)


    def testing(self, dataset_testing) -> list: 
        predicted_out = []
        for i in range(dataset_testing.shape[0]):
            self.feedforwarding(dataset_testing.loc[i:i])
            predicted_out.append(n.layer_list[-1][0][0])

        return predicted_out

    def feedforwarding(self, sample):
        '''fase di feedforwarding'''
        self.layer_list[0] = np.array(sample) #inizializzo l'input della rete ai valori del sample
        for layer_index in range(self.num_layers-1): # per ogni weight_matrix esistente
            out_unactivated = np.dot(self.layer_list[layer_index], self.weight_list[layer_index])#+0.25 #+ bias test
            #print(out_unactivated)
            self.layer_list[layer_index+1] = self.activation(out_unactivated)
            #print(self.layer_list)

    def backpropagation(self, i): 
        '''fase di backtracking'''
        
        #inizializzazione errore output
        error = -(self.activation(output_totale.loc[i:i])-self.layer_list[-1])
        self.derivate_list[-1] = error
        for j in range(num_layers-2, 0, -1): #3-2 layer: si parte dall'indice 1 con 0 escluso (es.)
            
            ad = self.activation_derivative(self.layer_list[j])
            summatory = np.matmul(self.weight_list[j], self.derivate_list[j])
            self.derivate_list[j-1] = np.multiply(ad, summatory.T).T
            '''aggiornamento dei pesi'''
            self.weight_list[j] -= self.learning_rate* np.outer(self.layer_list[j], self.derivate_list[j])

        #manca l'aggiornamento del peso all'indice 0 che non è calcolato nell'ultimo ciclo
        self.weight_list[0] -= learning_rate*np.outer(self.layer_list[0], self.derivate_list[0])

    def binary_cross_entropy(y_true, y_pred):
        epsilon = 1e-15  # Piccola costante per evitare divisioni per zero
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip dei valori per evitare valori di logaritmo negativi
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

        # Calcolo dell'errore per un singolo esempio di training o testing
        error = binary_cross_entropy(y_true, y_pred)

epochs = 1
learning_rate = 1
len_input = len(input_totale.columns)
num_layer_nodes = [len_input, 100, 1] #numero nodi di ciascun layer da sinistra verso destra
num_layers = len(num_layer_nodes) #numero di layer compresi nella rete neurale input e output compresi

n = NeuralNetwork(learning_rate=learning_rate, len_input=len_input, 
                num_layers=num_layers, num_layer_nodes=num_layer_nodes, epochs=epochs
                )
n.training(input_totale)

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

'''fase di testing'''

dataset_test = dataset_test.reset_index(drop=True)
X_test = dataset_test.loc[:, dataset_test.columns != output_string]


'''fase di testing'''
predicted_out = n.testing(X_test)
Y_effettivo = np.array(dataset_test[output_string])
real_out = np.array(1/(1+np.exp(-Y_effettivo)))


# Crea un'istanza di LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Converti le etichette di classe continue in etichette di classe discrete
output_totale_discrete = label_encoder.fit_transform(output_totale.values.ravel())

# Definisci e addestra il modello di rete neurale
model = MLPRegressor(hidden_layer_sizes=(100,100,), activation='logistic', solver='sgd', random_state=42)
model.fit(np.array(input_totale), np.array(output_totale_discrete))

# Esegui le previsioni sul test set
predictions = model.predict(X_test)

mse = np.mean((real_out-predictions)**2)
print("Mean Squared Error:", mse)





Epoca numero 0
Mean Squared Error: 0.05119007048411901


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [380]:
'''REGRESSION'''

output_string = 'Weight'
input_totale = dataset_training.loc[:len(dataset_training), dataset_training.columns != output_string]
output_totale = dataset_training.loc[:len(dataset_training), dataset_training.columns == output_string]

class NeuralNetwork:
    def __init__(self, learning_rate, len_input, num_layers, num_layer_nodes, epochs=1):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.len_input = len_input
        self.num_layers = num_layers
        self.num_layer_nodes = num_layer_nodes
        self.weight_list = []
        self.layer_list = []
        self.derivate_list = []
        self.threshold = 0.5
        self.totalerror = 0
        self.prediction = None #serve per il testing dell'errore


        #inizializzazione matrice dei pesi
        for i in range(self.num_layers-1):
            new_matrix = np.random.rand(self.num_layer_nodes[i], self.num_layer_nodes[i+1]) #matrice con righe=i e colonne i+1: (input,output) di un layer
            self.weight_list.append(new_matrix)

        #inizializzazione layer
        for i in range(self.num_layers):
            self.layer_list.append(np.zeros(self.num_layer_nodes[i]))

        #inizializzazione lista derivate parziali
        for i in range(self.num_layers-1): #non viene contato il layer di input
            self.derivate_list.append(np.zeros(self.num_layer_nodes[i+1]))


        #print("weight: \n", self.weight_list)
        #print("derivate: \n", self.derivate_list)
        #print("layer: \n". self.layer_list)


    def activation(self, x, type='sigmoid'): 
        if type=='sigmoid':
            return 1/(1+np.exp(-x))
        elif type=='tanh':
            return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        elif type=='relu':
            return (x>0)*x
        return None

    def activation_derivate(self, x, type='sigmoid'):
        if type=='sigmoid':
            return (1-self.activation(x, type='sigmoid'))*self.activation(x, type='sigmoid')
        elif type=='relu':
            return (x>0)*1
        return None
    
    def training(self, dataset_training, output):
        '''inizializzazione array output per il calcolo dell'errore'''
        self.prediction = np.zeros(dataset_training.shape[0])

        for e in range(self.epochs):
            #self.learning_rate = 1/(1000+self.epochs*20)
            print("Epoca numero", e, end='\t')
            for i in range(dataset_training.shape[0]): #ciclo che itera per ogni sample del dataset 
                sample = dataset_training.loc[i:i]
                self.feedforwarding(sample)
                '''salvataggio output per test errore'''
                self.prediction[i] = self.layer_list[-1][0][0]
                self.backpropagation(i)

            print("MSE: ", self.dataset_MSE(self.prediction, output))


    def testing(self, dataset_testing) -> list: 
        predicted_out = []
        for i in range(dataset_testing.shape[0]):
            self.feedforwarding(dataset_testing.loc[i:i])
            predicted_out.append(self.layer_list[-1][0][0])

        return predicted_out

    def feedforwarding(self, sample):
        '''fase di feedforwarding'''
        self.layer_list[0] = sample.values #inizializzo l'input della rete ai valori del sample
        for layer_index in range(self.num_layers-1): # per ogni weight_matrix esistente
            out_unactivated = np.dot(self.layer_list[layer_index], self.weight_list[layer_index])
            if layer_index < self.num_layers-2:
                self.layer_list[layer_index+1] = self.activation(out_unactivated)
                #print("Assigned layer list1: \n\n", self.layer_list[layer_index+1])
            else:
                self.layer_list[layer_index+1] = out_unactivated
                #print("Assigned layer list2: \n\n", self.layer_list[layer_index+1])

    def backpropagation(self, i): 
        '''fase di backtracking'''

        error = -(output_totale.loc[i:i]-self.layer_list[-1]).values 
        self.derivate_list[-1] = error*self.activation_derivate(self.layer_list[-1])
        
        for j in range(num_layers-2, 0, -1): #3-2 layer: si parte dall'indice 1 con 0 escluso (es.)
            ad = self.activation_derivate(self.layer_list[j])
            
            summatory = self.derivate_list[j]@self.weight_list[j].T #matmul o dot?
            self.derivate_list[j-1] = ad*summatory
            '''aggiornamento dei pesi'''
            self.weight_list[j] -= self.learning_rate*np.matmul(self.layer_list[j].T, self.derivate_list[j])
    
        self.weight_list[0] -= self.learning_rate*np.matmul(self.layer_list[0].T, self.derivate_list[0])
    
    def MSE(self, input, prediction):
        return (input-prediction)**2
    
    def dataset_MSE(self, prediction, output):
        return np.mean(self.MSE(prediction, output))

epochs = 50
learning_rate = 0.01
len_input = len(input_totale.columns)
num_layer_nodes = [len_input, 100, 1] #numero nodi di ciascun layer da sinistra verso destra
num_layers = len(num_layer_nodes) #numero di layer compresi nella rete neurale input e output compresi


output = np.array(dataset_training.loc[:, dataset_test.columns == output_string])
n = NeuralNetwork(learning_rate=learning_rate, len_input=len_input, 
                num_layers=num_layers, num_layer_nodes=num_layer_nodes, epochs=epochs,
                )

n.training(input_totale, output)





Epoca numero 0	MSE:  1130.4675811285288
Epoca numero 1	MSE:  1073.6381259274124
Epoca numero 2	MSE:  987.4557759305452
Epoca numero 3	MSE:  819.3243175768932
Epoca numero 4	MSE:  537.1421371384573
Epoca numero 5	MSE:  359.07822874477625
Epoca numero 6	MSE:  248.12149973049557
Epoca numero 7	MSE:  151.61973915509282
Epoca numero 8	MSE:  73.92081128423021
Epoca numero 9	MSE:  13.678513928283516
Epoca numero 10	MSE:  8.770381118009023
Epoca numero 11	MSE:  5.600697959164512
Epoca numero 12	MSE:  3.5457896211278563
Epoca numero 13	MSE:  2.4199401855991978
Epoca numero 14	MSE:  1.946565499678596
Epoca numero 15	MSE:  1.7907173230382913
Epoca numero 16	MSE:  1.7536788131872372
Epoca numero 17	MSE:  1.7572240365234306
Epoca numero 18	MSE:  1.7738664392094594
Epoca numero 19	MSE:  1.7938072105206506
Epoca numero 20	MSE:  1.8134820748021185
Epoca numero 21	MSE:  1.8315896529734066
Epoca numero 22	MSE:  1.8476931566996806
Epoca numero 23	MSE:  1.8617235946332968
Epoca numero 24	MSE:  1.873785017

In [381]:
'''fase di testing'''

dataset_test = dataset_test.reset_index(drop=True)
X_test = dataset_test.loc[:, dataset_test.columns != output_string]


'''fase di testing'''
predicted_out = np.array(n.testing(X_test))
Y_effettivo = np.array(dataset_test[output_string])
#real_out = np.array(1/(1+np.exp(-Y_effettivo)))


print(max(predicted_out), min(predicted_out))
total_error = np.mean(n.MSE(predicted_out, Y_effettivo))
print(total_error)

1.8401444995343148 -1.7599733674122124
0.06757057412882364
